# Clipper Exercise

This tutorial will walk you through the process of starting Clipper, creating and querying a Clipper application, and deploying models to Clipper. In the first part of the demo, you will set up Clipper and create an application without involving any machine learning, demonstrating how a frontend developer or dev-ops engineer can set up and query Clipper without having to know anything about the machine-learning models involved.

As an example, this tutorial will walk you through creating an application that labels images as either pictures of birds or planes. You will use the CIFAR-10 dataset as the source of these images.

These images have already been downloaded and are available locally at `/data/cifar`.

## Load Cifar

The first step in building any application, using machine-learning or otherwise, is to understand the application requirements. Load the dataset into the notebook so you can examine it and better understand the dataset you will be working with. The `cifar_utils` library provides several utilities for working with CIFAR data – we will make use of one of them here.

In [1]:
import cifar_utils

cifar_loc = "/data/cifar"
test_x, test_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=True))
no_norm_x, no_norm_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=False))

Take a look at the data you've loaded. The size and blurriness of these photos should give you a better understanding of the difficulty of the task you will ask of your machine learning models! If you'd like to see more images, increase the number of rows of images displayed -- the last argument to the function -- to a number greater than 2.

In [ ]:
%matplotlib inline
cifar_utils.show_example_images(no_norm_x, no_norm_y, 2)

# Start Clipper

Now you're ready to start Clipper! You will be using the `clipper_admin` client library to perform administrative commands.

In [ ]:
import sys
import os
from clipper_admin import Clipper
host = ""
clipper_conn = Clipper("localhost")

clipper_conn.start()

Congratulations! You now have a running Clipper instance that you can start to interact with. Think of your `clipper` Python object as a vehicle for that interaction. Try using it to see the applications deployed to this Clipper instance:

In [ ]:
clipper.get_all_apps()

# Create an application

In order to query Clipper for predictions, you need to create an application. Each application specifies a name, the query input datatype, the selection policy, and a latency service level objective. Once you register an application with Clipper, the system will create a REST endpoint for handling prediction requests.

By associating the query interface with a specific application, Clipper allows frontend developers the flexibility to have multiple applications running in the same Clipper instance. Applications can request predictions from any model in Clipper. This allows a single Clipper instance to serve multiple machine-learning applications. It also provides a convenient mechanism for beta-testing or incremental rollout by creating experimental and stable applications for the same set of queries.

For this tutorial, you will create an application named "cifar_demo". Note that Clipper allows you to create the application before deploying the models. Clipper will be moving to a label-based model specification mechanism soon, so that in the future you won't have to explicitly enumerate all the models you want to query up front.

Registering the `cifar-demo` application with Clipper will have the following effect on your setup: <img src="img/register_app.png" style="width: 500px;"/>

> *Don't worry if this command seems to take a long time. Before starting Clipper, the Docker containers must be downloaded from Docker Hub. These containers are fairly large and may take awhile to download depending on the speed of your internet connection.*

In [ ]:
app_name = "cifar-demo"
# If the model (which we will later link to our application) doesn't
# return a prediction in time, predict label 0 (bird) by default
default_output = "0"

clipper.register_application(
    app_name,
    "doubles",
    default_output,
    slo_micros=20000)

Now when you list the applications registered with Clipper, you should see the newly registered "cifar-demo" application show up!

In [ ]:
clipper.get_all_apps(verbose=True)

# Start serving

Now that you have registered an application, you can start querying the application for predictions. In this case,
Clipper has created a REST endpoint for your application at:
```
http://localhost:1337/cifar_demo/predict
```

Try using cURL to query


You will now start querying Clipper with a simple Python frontend app that computes the average accuracy of the responses after every 100 requests and updates a plot of the results with every iteration.

This diagram shows how the accuracy plot is receiving its test predictions: <img src="img/serve_predictions.png" style="width: 500px;"/>